In [1]:
import whisper
import torch

from utils import log_mel_spectrogram, load_audio, pad_or_trim, N_FRAMES
from whisper.decoding import DecodingOptions
from model import Whisper
from whisper.tokenizer import get_tokenizer


model_original = whisper.load_model("tiny")
model_original.to(torch.float32)

tokenizer = get_tokenizer(model_original.is_multilingual, language='en', task='transcribe')

model = Whisper.load_from_pretrained("tiny") 

/Users/alex/.pyenv/versions/3.10.6/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


In [2]:
mel = log_mel_spectrogram(load_audio('./audio.mp3'))
mel_segment = pad_or_trim(mel, N_FRAMES).unsqueeze(0)
model_original.decode(mel_segment, DecodingOptions(fp16=False))[0].text

'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the exhibition.'

In [3]:
mel.shape

torch.Size([80, 965])

In [4]:
original_encoder_logits = model_original.encoder(mel_segment)
encoder_logits = model.encoder(mel_segment)

assert torch.allclose(original_encoder_logits, encoder_logits)

tokens = torch.randint(0, 50000, (1,15))
original_decoder_logits = model_original.decoder(tokens, original_encoder_logits)
decoder_logits = model.decoder(tokens, encoder_logits)

assert torch.allclose(original_decoder_logits, decoder_logits)

encoder_logits = model.encoder(mel_segment)
tokenizer.decode(model.generate(torch.tensor([tokenizer.sot_sequence], dtype=torch.int64), encoder_logits)[0,4:]).strip()

'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the exhibition.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [5]:
# Datasets
# Layer 1 -> Audio Dataset
# Layer 2 -> Mel Spectrogram Dataset

In [6]:
from dataset import AudioDatasetFake, SpectrogramDataset, Batch
dataset = SpectrogramDataset(AudioDatasetFake())
len(list(dataset))

5

In [8]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=2, num_workers=2, collate_fn=Batch.collate_fn)


In [10]:
for i in dataloader:
    print(i)

Batch(inputs=tensor([[[-0.5596, -0.5596, -0.4869,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5596, -0.2426,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5387, -0.1150,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.5596, -0.5596,  0.0367,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5068,  0.2271,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5566,  0.0240,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.5596, -0.5596, -0.4869,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5596, -0.2426,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5387, -0.1150,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.5596, -0.5596,  0.0367,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5068,  0.2271,  ...,  0.0000,  0.0000,  0.0000],
         [-0.5596, -0.5566,  0.0240,  ...,  0.0000,  0.0000,  0.0000]]]), ids=[0, 2], max_dense_tokens=16)
Batch(inputs=tensor([[[-0.5596, -0.5596, -0.4869,  ...,  0.0000,  0.0